In [7]:
import os
import pandas as pd
import numpy as np

In [8]:
# Read in vehicle crash information
df = pd.read_csv('Crashes_in_DC.csv')

C:\Users\Frank\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
bicyclist_pedestrian_index = ['MAJORINJURIES_BICYCLIST','MINORINJURIES_BICYCLIST', 'FATAL_BICYCLIST',\
                             'MAJORINJURIES_PEDESTRIAN','MINORINJURIES_PEDESTRIAN','FATAL_PEDESTRIAN']

# Keep only incidents involving bicyclists or pedestrians
df = df.loc[(df['MAJORINJURIES_BICYCLIST']==1) | (df['MINORINJURIES_BICYCLIST']==1) | \
            (df['FATAL_BICYCLIST']==1) | (df['MAJORINJURIES_PEDESTRIAN']==1) | \
            (df['MINORINJURIES_PEDESTRIAN']==1) | (df['FATAL_PEDESTRIAN']==1)]

# Convert dates to datime format
df['REPORTDATE'] = pd.to_datetime(df['REPORTDATE'])

In [10]:
# Calculate number of incidents occuring each date incidents according to date

bicyclist_pedestrian_index = ['MAJORINJURIES_BICYCLIST','MINORINJURIES_BICYCLIST', 'FATAL_BICYCLIST',\
                             'MAJORINJURIES_PEDESTRIAN','MINORINJURIES_PEDESTRIAN','FATAL_PEDESTRIAN']

dates = pd.date_range(start='2012-01-01', end='2018-10-01', freq='M')
dates[0].year

incidents = np.zeros((len(dates),len(bicyclist_pedestrian_index)))

for i in range(len(dates)):
    start = str(dates[i].year)+'-'+str(dates[i].month)+'-01'
    end = str(dates[i].year)+'-'+str(dates[i].month)+'-'+str(dates[i].day)
    
    mask = (df['REPORTDATE'] >= start) & (df['REPORTDATE'] < end)
    incidents[i,:] = df[bicyclist_pedestrian_index].loc[mask].sum().values
 
df_incidents_date = pd.DataFrame(incidents,index=dates,columns = bicyclist_pedestrian_index)

In [11]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool

output_notebook()
# Visualize datetime of incidents

df_incidents_date['Date'] = df_incidents_date.index.values
df_incidents_date['Total Incidents'] = df_incidents_date.sum(axis=1)

source_time = ColumnDataSource(df_incidents_date)

hover = HoverTool(tooltips=[
                            ('date', '@Date{%F}'),
                            ('minor ped. injuries', '@MINORINJURIES_PEDESTRIAN{%d}'),
                            ('major ped. injuries', '@MAJORINJURIES_PEDESTRIAN{%d}'),
                            ('ped. fatalities', '@FATAL_PEDESTRIAN{%d}'),
                            ('minor bic. injuries', '@MINORINJURIES_BICYCLIST{%d}'),
                            ('major bic. injuries', '@MAJORINJURIES_BICYCLIST{%d}'),
                            ('bic. fatalities', '@FATAL_BICYCLIST{%d}')
                           ],
          formatters={
              'Date': 'datetime',
              'MINORINJURIES_PEDESTRIAN' : 'printf',
              'MAJORINJURIES_PEDESTRIAN' : 'printf',
              'FATAL_PEDESTRIAN' : 'printf',
              'MINORINJURIES_BICYCLIST' : 'printf',
              'MAJORINJURIES_BICYCLIST' : 'printf',
              'FATAL_BICYCLIST' : 'printf',
          },
        mode='vline')

p = figure(x_axis_type="datetime", title="Number of incidents per Month", plot_height=350, plot_width=950,\
           tools=[hover,'wheel_zoom','pan','reset'])

p.line(x = 'Date', y = 'Total Incidents', line_width=2, color="firebrick", source = source_time)

#p.line(df_incidents_date.index, df_incidents_date[bicyclist_pedestrian_index[:4]].sum(axis=1), \
#       line_width=2, color="firebrick")
#p.line(df_incidents_date.index, df_incidents_date[bicyclist_pedestrian_index[4:]].sum(axis=1), \
#       line_width=2, color="navy")

show(p)

Loading BokehJS ...